In [1129]:
import os
import random
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
pd.options.display.float_format = '{:.5f}'.format


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42)

train = pd.read_csv('/Users/idogeon/Desktop/현대/데이터/train.csv')
test = pd.read_csv('/Users/idogeon/Desktop/현대/데이터/test.csv')

train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR
0,TRAIN_000000,CN,EKP8,Bulk,30.73658,2020-10-15 4:03,Z517571,30.00000,28,73100,...,3.77000,15.90000,2.73080,12,42.01000,43.16000,40.96000,1407.66833,0.00166,3.04833
1,TRAIN_000001,CN,EUC8,Container,63.22042,2019-09-17 2:55,U467618,30.00000,15,37900,...,-6.72000,24.50000,4.28906,10,67.53000,64.55000,59.34000,2089.04677,0.00161,17.13861
2,TRAIN_000002,CN,NGG6,Container,90.42742,2019-02-23 6:43,V378315,50.00000,7,115000,...,0.00000,9.40000,0.00000,14,65.30000,66.39000,56.94000,603.19305,0.00174,98.82750
3,TRAIN_000003,JP,TMR7,Cargo,0.00000,2020-09-18 22:06,B726632,10.00000,33,1490,...,-7.31000,22.10000,4.69374,7,43.02000,43.15000,41.11000,1169.85345,0.00007,0.00000
4,TRAIN_000004,RU,NNC2,Container,8.81373,2022-08-13 12:57,D215135,30.00000,10,27600,...,2.31000,22.80000,2.34587,14,90.45000,93.65000,88.11000,1107.94489,0.00020,96.03056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367436,TRAIN_367436,CN,YRT6,Bulk,59.01818,2017-11-11 22:23,J661243,40.00000,13,93200,...,NaN,NaN,NaN,6,61.25000,62.21000,55.70000,1333.60911,0.00036,65.85000
367437,TRAIN_367437,JP,QYY1,Tanker,0.00000,2022-04-29 2:58,D847216,10.00000,9,1280,...,0.87000,17.10000,1.02856,11,105.37000,109.34000,104.69000,1955.10385,0.00055,0.00000
367438,TRAIN_367438,SG,GIW5,Container,1.76863,2022-07-14 7:58,Q635545,30.00000,6,25000,...,3.36000,31.70000,2.55716,15,97.73000,99.10000,95.78000,1601.29109,0.00262,0.99750
367439,TRAIN_367439,JP,TMR7,Cargo,0.00000,2020-12-22 10:07,N211282,10.00000,8,2400,...,-2.44000,10.80000,3.05572,19,49.75000,50.08000,47.02000,1191.35333,0.00007,0.00000


In [1130]:
train.describe()

,DIST,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR
count,367441.00000,367440.00000,367441.00000,367441.00000,367440.00000,367440.00000,367441.00000,367440.00000,213955.00000,213955.00000,213076.00000,213955.00000,367441.00000,367441.00000,367441.00000,367441.00000,367441.00000,367441.00000,367441.00000
mean,19.31593,30.55024,14.74608,63588.86706,16.47654,11.08671,41346.60661,205.11447,-0.29759,-0.26296,18.86734,2.70648,11.83790,65.53381,67.55712,63.20949,1348.25133,0.00088,61.87712
std,30.90327,10.95744,7.04039,63558.70531,6.04701,4.60002,39311.57900,75.86738,3.36591,3.73511,9.72709,1.38763,6.64567,20.91664,20.98516,20.63043,738.96105,0.00084,170.57522
min,0.00000,10.00000,0.00000,100.00000,0.00000,0.00000,150.00000,20.00000,-25.33000,-21.45000,-32.20000,0.00000,0.00000,13.52000,19.33000,-37.63000,286.80230,0.00000,0.00000
25%,0.00000,20.00000,10.00000,14400.00000,10.00000,10.00000,10000.00000,150.00000,-2.15000,-2.28000,12.20000,1.80429,7.00000,50.02000,51.46000,48.36000,836.34920,0.00014,0.00000
50%,6.92993,30.00000,13.00000,47400.00000,20.00000,10.00000,30700.00000,190.00000,-0.13000,0.00000,21.60000,2.61806,12.00000,62.93000,64.62000,59.14000,1187.57373,0.00055,7.94944
75%,25.69306,40.00000,18.00000,81400.00000,20.00000,10.00000,52100.00000,260.00000,1.54000,2.00000,26.60000,3.55601,17.00000,76.88000,79.30000,74.25000,1660.98762,0.00161,49.15333
max,199.98065,60.00000,80.00000,404000.00000,30.00000,20.00000,237000.00000,400.00000,17.91000,29.69000,47.60000,11.17966,23.00000,127.86000,127.98000,123.70000,4795.86561,0.00262,2159.13056


## 중복없음

In [1132]:
train[train.duplicated(keep=False)]

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR


### 데이터 정의
- ARI_CO = 도착항의 소속국가(도착항 앞 2글자) - 범주형
- ARI_PO = 도착항의 항구명 (도착항 뒤 글자) 범주형
- SHIP_TYPE_CATEGORY = 선종 통합 바탕으로 5대 선종으로 분류(4개) - 범주형
- DIST = 정박지(ber_port)와 접인지 사이의 거리 - 연속형
- ATA = anc_port에 도착한 시점의 utc.실제 정박 시각(Actual Time of Arrival) - datetime
- BREADTH = 선박의 폭 - 범주형
- BUILT = 선박의 연령 - 범주형
- DEADWEIGHT = 선박의 재화중량톤수 - 연속형
- DEPTH = 선박의 깊이 - 범주형
- DRAUGHT = 홀수 높이 - 범주형
- GT = 용적톤수(Gross Tonnage)값 - 연속형
- LENGTH = 선박의 길이 - 연속형
- SHIPMANAGER = 선박 소유주 - 범주형
- FLAG = 선박의 국적 - 범주형
- U_WIND = 풍향 u벡터 - 연속형
- V_WIND = 풍향 v벡터 - 연속형
- AIR_TEMPERATURE = 기온 - 연속형
- BN = 보퍼트 풍력 계급 - 범주형
- ATA_LT = anc_port에 도착한 시점의 현지 정박 시각(Local Time of Arrival)(단위 : H) - 연속형
- DUBAi = 해당일의 두바이유 값 - 연속형
- BRENT = 해당일의 브렌트유 값 - 연속형
- WTI = 해당일의 WTI 값 - 연속형
- BDI_ADJ = 조정된 벌크운임지수 - 연속형
- PORT_SIZE = 접인지 폴리곤 영역의 크기 - 연속형

## 기상관련변수
- U_WIND (풍속)
- V_WIND (풍속)
- ATA변수 (year, month, hour, day, minute, weekday,weekend)
- wind_speed 
- BN (B) 
- B
- AIR_TEMPERATURE

## 지리관련변수
- DIST 

## 선박관련변수
- ID
- BREADTH
- DEADWEIGHT
- BUILT
- DEPTH
- DRAUGHT
- GT
- LENGTH
- SHIPMANAGER
- FLAG
- BDI_ADJ
- PORT_SIZE
- SHIP_TYPE_CATEGORY
- CI_HOUR (CI)

## 항구관련함수
- ARI_CO
- ARI_PO
- PORT_SIZE

## 가격관련함수
- BDI_ADJ
- DUBAI
- BREENT
- WTI

### ATA변수 datetime으로 변환 후 year, month, day, hour, minute, weekday 파생변수 생성
### 'BERADTH', 'DEPTH', 'DRAUGHT', 'LENGTH' 결측치는 데이터 하나에 존재 -> 제거

In [1133]:
train.drop(23856,axis=0,inplace=True)

## 노이즈 데이터 
- utc 시간 기준으로 같은 국가, 같은 항구, 같은 시간일 때 풍향, 기온, 유가 가격이 다른 데이터 제거
- 혹은 결측치로 대체 후 결측치와 함께 처리

In [1134]:
# 'ARI_CO', 'ARI_PO', 'ATA' 별로 'DUBAI', 'BRENT', 'WTI' 열의 고유한 값 수를 계산합니다.
unique_counts = train.groupby(['ARI_CO', 'ARI_PO', 'ATA'])[['U_WIND','V_WIND','AIR_TEMPERATURE','DUBAI','BRENT','WTI']].nunique()
# 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE' 열 중에서 고유한 값 수가 1보다 큰 행을 필터링합니다.
filtered_data = unique_counts[(unique_counts['DUBAI'] > 1) |(unique_counts['BRENT'] > 1) |(unique_counts['WTI'] > 1)|
                             (unique_counts['U_WIND'] > 1) |(unique_counts['V_WIND'] > 1) |(unique_counts['AIR_TEMPERATURE'] > 1)] 
# 고유한 값 수가 1보다 큰 그룹의 인덱스 추출
filtered_indices = filtered_data.index
# 'ARI_CO', 'ARI_PO', 'ATA'가 필터링된 인덱스에 포함되지 않는 행만 선택하여 새로운 데이터프레임 생성
train = train[~train.set_index(['ARI_CO', 'ARI_PO', 'ATA']).index.isin(filtered_indices)]

# ATA 
- ATA datetime으로 변환
- year, month, day, minute,weekday 파생변수 생성

In [1135]:
# 원본 데이터프레임을 복사하여 새로운 데이터프레임을 생성
new_train = train.copy()
# 새로운 데이터프레임에 작업 수행
new_train['ATA'] = pd.to_datetime(new_train['ATA'])
# 새로운 데이터프레임을 원본 데이터프레임에 할당
train = new_train

# datetime을 여러 파생 변수로 변환
# 토요일5, 일요일 6
for df in [train]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday
    

In [1136]:
# 원본 데이터프레임을 복사하여 새로운 데이터프레임을 생성
new_test = test.copy()
# 새로운 데이터프레임에 작업 수행
new_test['ATA'] = pd.to_datetime(new_test['ATA'])
# 새로운 데이터프레임을 원본 데이터프레임에 할당
test = new_test
# datetime을 여러 파생 변수로 변환
for df in [test]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday

# 결측치 처리
#### 20년 이후 데이터는 결측치 없음. 18년 이전데이터는 모두 결측치 . 18년 19년은 있는 것도 있고 없는 것도 있음.
- MIce

## 인코딩 먼저해야 Mice 가능

In [1137]:
X = train.copy()

In [1138]:
import pandas as pd

# 'SHIP_TYPE_CATEGORY' 변수에 대해 원핫인코딩 적용
X = pd.get_dummies(X, columns=['SHIP_TYPE_CATEGORY'],drop_first=True)
test = pd.get_dummies(test, columns=['SHIP_TYPE_CATEGORY'],drop_first=True)
#X_val = pd.get_dummies(X_val, columns=['SHIP_TYPE_CATEGORY'],drop_first=True)
#X_train


In [1139]:
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import bisect
# Categorical 컬럼 인코딩
categorical_features = ['ARI_CO', 'ARI_PO', 'ID', 'SHIPMANAGER', 'FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
    le = LabelEncoder()
    X[feature] = le.fit_transform(X[feature].astype(str))
    le_classes_set = set(le.classes_)
    test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
    le_classes = le.classes_.tolist()
    bisect.insort_left(le_classes, '-1')
    le.classes_ = np.array(le_classes)
    test[feature] = le.transform(test[feature].astype(str))
    encoders[feature] = le

Encoding features: 100%|██████████████████████████| 5/5 [00:01<00:00,  3.49it/s]


In [1140]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# IterativeImputer 모델 생성
imputer = IterativeImputer(max_iter=10, random_state=0)

# 'ARI_CO', 'ARI_PO', 'year', 'month', 'hour', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE' 열을 포함한 열들 선택
selected_columns = ['ARI_CO', 'ARI_PO', 'year', 'month', 'hour', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE']
data_for_imputation = X[selected_columns]

# IterativeImputer를 사용하여 결측치 채우기
imputed_data = imputer.fit_transform(data_for_imputation)

# 결과를 기존 데이터프레임에 적용
X[selected_columns] = imputed_data

# train 데이터프레임에 결측치가 없는지 확인
X.isnull().sum()


SAMPLE_ID                            0
ARI_CO                               0
ARI_PO                               0
DIST                                 0
ATA                                  0
ID                                   0
BREADTH                              0
BUILT                                0
DEADWEIGHT                           0
DEPTH                                0
DRAUGHT                              0
GT                                   0
LENGTH                               0
SHIPMANAGER                          0
FLAG                                 0
U_WIND                               0
V_WIND                               0
AIR_TEMPERATURE                      0
BN                              152015
ATA_LT                               0
DUBAI                                0
BRENT                                0
WTI                                  0
BDI_ADJ                              0
PORT_SIZE                            0
CI_HOUR                  

In [1141]:
train = X.copy()

In [1142]:
# 'ARI_CO', 'ARI_PO', 'year', 'month', 'hour', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE' 열을 포함한 열들 선택
selected_columns = ['ARI_CO', 'ARI_PO', 'year', 'month', 'hour', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE']
data_for_imputation = test[selected_columns]

# IterativeImputer를 사용하여 결측치 채우기
imputed_data = imputer.transform(data_for_imputation)

# 결과를 기존 데이터프레임에 적용
test[selected_columns] = imputed_data

# train 데이터프레임에 결측치가 없는지 확인
test.isnull().sum()


SAMPLE_ID                            0
ARI_CO                               0
ARI_PO                               0
DIST                                 0
ATA                                  0
ID                                   0
BREADTH                              0
BUILT                                0
DEADWEIGHT                           0
DEPTH                                0
DRAUGHT                              0
GT                                   0
LENGTH                               0
SHIPMANAGER                          0
FLAG                                 0
U_WIND                               0
V_WIND                               0
AIR_TEMPERATURE                      0
BN                              101927
ATA_LT                               0
DUBAI                                0
BRENT                                0
WTI                                  0
BDI_ADJ                              0
PORT_SIZE                            0
year                     

# 파생변수 생성식

## 풍속계산

In [1143]:
import numpy as np
train['wind_speed'] = np.sqrt(train['U_WIND']**2 + train['V_WIND']**2)
test['wind_speed'] = np.sqrt(test['U_WIND']**2 + test['V_WIND']**2)

# 보퍼트 계급 식 : 풍속 = 0.836 * (BN) **(3/2)

In [1144]:
# 'WIND_SPEED'를 기반으로 'BN' 변수의 결측치 채우기
train['BN'].fillna((train['wind_speed'] / 0.836) ** (2/3), inplace=True)
test['BN'].fillna((test['wind_speed'] / 0.836) ** (2/3), inplace=True)
# 결과 확인
train.isnull().sum()


SAMPLE_ID                       0
ARI_CO                          0
ARI_PO                          0
DIST                            0
ATA                             0
ID                              0
BREADTH                         0
BUILT                           0
DEADWEIGHT                      0
DEPTH                           0
DRAUGHT                         0
GT                              0
LENGTH                          0
SHIPMANAGER                     0
FLAG                            0
U_WIND                          0
V_WIND                          0
AIR_TEMPERATURE                 0
BN                              0
ATA_LT                          0
DUBAI                           0
BRENT                           0
WTI                             0
BDI_ADJ                         0
PORT_SIZE                       0
CI_HOUR                         0
year                            0
month                           0
day                             0
hour          

# ATA변수 현지시간으로 바꾸기 
- year, month, day, minute,weekday 파생변수 생성
- ATA_LT를 hour값으로 사용 -> 왜냐하면 같은 항구지만 시간차가 다른경우가 존재. 1~2시간 차이기 때문에 hour은 ATA_LT로 기존 변수의 시간으로 하기로함
- weekend 파생변수 생성 -> weekday값이 5,6 이면 weekend = 1 else = 0.

In [1145]:
conditions = [
    (train['ARI_CO'] == 'CN'),  # 'CN'인 경우 8시간 추가
    (train['ARI_CO'] == 'JP'),  # 'JP'인 경우 9시간 추가
    (train['ARI_CO'] == 'RU'),  # 'RU'인 경우 2시간 추가
    (train['ARI_CO'] == 'AU'),  # 'AU'인 경우 11시간 추가
    (train['ARI_CO'] == 'SG'),  # 'SG'인 경우 8시간 추가
    (train['ARI_CO'] == 'ZA'),  # 'ZA'인 경우 2시간 추가
    (train['ARI_CO'] == 'KR'),  # 'KR'인 경우 9시간 추가
    (train['ARI_CO'] == 'TW'),  # 'TW'인 경우 8시간 추가
    (train['ARI_CO'] == 'TT'),  # 'TT'인 경우 -4시간 추가
    (train['ARI_CO'] == 'ID'),  # 'ID'인 경우 7시간 추가
    (train['ARI_CO'] == 'BR'),  # 'BR'인 경우 -2시간 추가
    (train['ARI_CO'] == 'QA'),  # 'QA'인 경우 3시간 추가
    (train['ARI_CO'] == 'LV'),  # 'LV'인 경우 3시간 추가
    (train['ARI_CO'] == 'MZ'),  # 'MZ'인 경우 2시간 추가
    (train['ARI_CO'] == 'US'),  # 'US'인 경우 -4시간 추가
    (train['ARI_CO'] == 'IN'),  # 'IN'인 경우 5.5시간 추가
    (train['ARI_CO'] == 'UA'),  # 'UA'인 경우 3시간 추가
    (train['ARI_CO'] == 'CA'),  # 'CA'인 경우 -2.5시간 추가
    (train['ARI_CO'] == 'MY'),  # 'MY'인 경우 8시간 추가
    (train['ARI_CO'] == 'PE'),  # 'PE'인 경우 -5시간 추가
    (train['ARI_CO'] == 'VN'),  # 'VN'인 경우 7시간 추가
    (train['ARI_CO'] == 'FI'),  # 'FI'인 경우 3시간 추가
    (train['ARI_CO'] == 'CL'),  # 'CL'인 경우 -3시간 추가
    (train['ARI_CO'] == 'VE'),  # 'VE'인 경우 -4시간 추가
    (train['ARI_CO'] == 'PH')   # 'PH'인 경우 8시간 추가
]

choices = [
    train['ATA'] + pd.Timedelta(hours=8),
    train['ATA'] + pd.Timedelta(hours=9),
    train['ATA'] + pd.Timedelta(hours=2),
    train['ATA'] + pd.Timedelta(hours=11),
    train['ATA'] + pd.Timedelta(hours=8),
    train['ATA'] + pd.Timedelta(hours=2),
    train['ATA'] + pd.Timedelta(hours=9),
    train['ATA'] + pd.Timedelta(hours=8),
    train['ATA'] + pd.Timedelta(hours=-4),
    train['ATA'] + pd.Timedelta(hours=7),
    train['ATA'] + pd.Timedelta(hours=-2),
    train['ATA'] + pd.Timedelta(hours=3),
    train['ATA'] + pd.Timedelta(hours=3),
    train['ATA'] + pd.Timedelta(hours=2),
    train['ATA'] + pd.Timedelta(hours=-4),
    train['ATA'] + pd.Timedelta(hours=5.5),
    train['ATA'] + pd.Timedelta(hours=3),
    train['ATA'] + pd.Timedelta(hours=-2.5),
    train['ATA'] + pd.Timedelta(hours=8),
    train['ATA'] + pd.Timedelta(hours=-5),
    train['ATA'] + pd.Timedelta(hours=7),
    train['ATA'] + pd.Timedelta(hours=3),
    train['ATA'] + pd.Timedelta(hours=-3),
    train['ATA'] + pd.Timedelta(hours=-4),
    train['ATA'] + pd.Timedelta(hours=8)
]

train['ATA'] = np.select(conditions, choices, default=train['ATA'])

# datetime을 여러 파생 변수로 변환
# 토요일5, 일요일 6
for df in [train]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday
    
train['hour']=train['ATA_LT']
train.drop('ATA_LT',axis=1,inplace=True)
train['weekend'] = train['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)

In [1146]:
test['ATA'] = pd.to_datetime(test['ATA'])

conditions = [
    (test['ARI_CO'] == 'CN'),  # 'CN'인 경우 8시간 추가
    (test['ARI_CO'] == 'JP'),  # 'JP'인 경우 9시간 추가
    (test['ARI_CO'] == 'RU'),  # 'RU'인 경우 2시간 추가
    (test['ARI_CO'] == 'AU'),  # 'AU'인 경우 11시간 추가
    (test['ARI_CO'] == 'SG'),  # 'SG'인 경우 8시간 추가
    (test['ARI_CO'] == 'ZA'),  # 'ZA'인 경우 2시간 추가
    (test['ARI_CO'] == 'KR'),  # 'KR'인 경우 9시간 추가
    (test['ARI_CO'] == 'TW'),  # 'TW'인 경우 8시간 추가
    (test['ARI_CO'] == 'TT'),  # 'TT'인 경우 -4시간 추가
    (test['ARI_CO'] == 'ID'),  # 'ID'인 경우 7시간 추가
    (test['ARI_CO'] == 'BR'),  # 'BR'인 경우 -2시간 추가
    (test['ARI_CO'] == 'QA'),  # 'QA'인 경우 3시간 추가
    (test['ARI_CO'] == 'LV'),  # 'LV'인 경우 3시간 추가
    (test['ARI_CO'] == 'MZ'),  # 'MZ'인 경우 2시간 추가
    (test['ARI_CO'] == 'US'),  # 'US'인 경우 -4시간 추가
    (test['ARI_CO'] == 'IN'),  # 'IN'인 경우 5.5시간 추가
    (test['ARI_CO'] == 'UA'),  # 'UA'인 경우 3시간 추가
    (test['ARI_CO'] == 'CA'),  # 'CA'인 경우 -2.5시간 추가
    (test['ARI_CO'] == 'MY'),  # 'MY'인 경우 8시간 추가
    (test['ARI_CO'] == 'PE'),  # 'PE'인 경우 -5시간 추가
    (test['ARI_CO'] == 'VN'),  # 'VN'인 경우 7시간 추가
    (test['ARI_CO'] == 'FI'),  # 'FI'인 경우 3시간 추가
    (test['ARI_CO'] == 'CL'),  # 'CL'인 경우 -3시간 추가
    (test['ARI_CO'] == 'VE'),  # 'VE'인 경우 -4시간 추가
    (test['ARI_CO'] == 'PH')   # 'PH'인 경우 8시간 추가
]

choices = [
    test['ATA'] + pd.Timedelta(hours=8),
    test['ATA'] + pd.Timedelta(hours=9),
    test['ATA'] + pd.Timedelta(hours=2),
    test['ATA'] + pd.Timedelta(hours=11),
    test['ATA'] + pd.Timedelta(hours=8),
    test['ATA'] + pd.Timedelta(hours=2),
    test['ATA'] + pd.Timedelta(hours=9),
    test['ATA'] + pd.Timedelta(hours=8),
    test['ATA'] + pd.Timedelta(hours=-4),
    test['ATA'] + pd.Timedelta(hours=7),
    test['ATA'] + pd.Timedelta(hours=-2),
    test['ATA'] + pd.Timedelta(hours=3),
    test['ATA'] + pd.Timedelta(hours=3),
    test['ATA'] + pd.Timedelta(hours=2),
    test['ATA'] + pd.Timedelta(hours=-4),
    test['ATA'] + pd.Timedelta(hours=5.5),
    test['ATA'] + pd.Timedelta(hours=3),
    test['ATA'] + pd.Timedelta(hours=-2.5),
    test['ATA'] + pd.Timedelta(hours=8),
    test['ATA'] + pd.Timedelta(hours=-5),
    test['ATA'] + pd.Timedelta(hours=7),
    test['ATA'] + pd.Timedelta(hours=3),
    test['ATA'] + pd.Timedelta(hours=-3),
    test['ATA'] + pd.Timedelta(hours=-4),
    test['ATA'] + pd.Timedelta(hours=8)
]

test['ATA'] = np.select(conditions, choices, default=test['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [test]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday
    
    
test['hour']=test['ATA_LT']
test.drop('ATA_LT',axis=1,inplace=True)
test['weekend'] = test['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)

In [1147]:
# 'day' 변수가 1~10인 경우 '초순', 11~20인 경우 '중순', 나머지 경우 '하순'으로 변경
train['day'] = train['day'].apply(lambda x: '초순' if x in range(1, 11) else ('중순' if x in range(11, 21) else '하순'))
test['day'] = test['day'].apply(lambda x: '초순' if x in range(1, 11) else ('중순' if x in range(11, 21) else '하순'))

In [1148]:
# 'day' 변수에 대해 원핫인코딩 적용
train = pd.get_dummies(train, columns=['day'],drop_first=True)
test = pd.get_dummies(test, columns=['day'],drop_first=True)

In [1149]:

# 'wind_speed' 값을 기반으로 'B' 변수를 만들기 위한 함수 정의
def categorize_wind_speed(speed):
    if speed < 0.3:
        return 0
    elif 0.3 <= speed < 1.5:
        return 1
    elif 1.5 <= speed < 3.3:
        return 2
    elif 3.3 <= speed < 5.5:
        return 3
    elif 5.5 <= speed < 8:
        return 4
    elif 8 <= speed < 10.8:
        return 5
    elif 10.8 <= speed < 13.9:
        return 6
    elif 13.9 <= speed < 17.2:
        return 7
    elif 17.2 <= speed < 20.7:
        return 8
    elif 20.7 <= speed < 24.5:
        return 9
    elif 24.5 <= speed < 28.4:
        return 10
    elif 28.4 <= speed < 32.6:
        return 11
    else:
        return 12

# 'wind_speed'를 기반으로 'B' 변수 생성
train['B'] = train['wind_speed'].apply(categorize_wind_speed)
# 결과 출력
train


,SAMPLE_ID,ARI_CO,ARI_PO,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,...,minute,weekday,SHIP_TYPE_CATEGORY_Cargo,SHIP_TYPE_CATEGORY_Container,SHIP_TYPE_CATEGORY_Tanker,wind_speed,weekend,day_초순,day_하순,B
0,TRAIN_000000,4.00000,13.00000,30.73658,2020-10-15 04:03:00,24205,30.00000,28,73100,20.00000,...,3,3,0,0,0,3.77260,0,0,0,3
1,TRAIN_000001,4.00000,14.00000,63.22042,2019-09-17 02:55:00,19450,30.00000,15,37900,20.00000,...,55,1,0,1,0,7.42590,0,0,0,4
2,TRAIN_000002,4.00000,45.00000,90.42742,2019-02-23 06:43:00,20345,50.00000,7,115000,20.00000,...,43,5,0,1,0,0.00000,1,0,1,0
3,TRAIN_000003,8.00000,75.00000,0.00000,2020-09-18 22:06:00,1680,10.00000,33,1490,10.00000,...,6,4,1,0,0,8.50128,0,0,0,5
4,TRAIN_000004,16.00000,46.00000,8.81373,2022-08-13 12:57:00,2973,30.00000,10,27600,10.00000,...,57,5,0,1,0,3.00375,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367434,TRAIN_367434,4.00000,62.00000,55.40877,2022-06-16 14:27:00,3021,30.00000,16,26500,10.00000,...,27,3,0,0,0,3.60999,0,0,0,3
367435,TRAIN_367435,0.00000,37.00000,0.00000,2014-11-10 23:04:00,21011,40.00000,11,87300,20.00000,...,4,0,0,0,0,1.03774,0,1,0,1
367436,TRAIN_367436,4.00000,100.00000,59.01818,2017-11-11 22:23:00,9121,40.00000,13,93200,20.00000,...,23,5,0,0,0,0.37773,1,0,0,1
367437,TRAIN_367437,8.00000,66.00000,0.00000,2022-04-29 02:58:00,3755,10.00000,9,1280,0.00000,...,58,4,0,0,1,0.87207,0,0,1,1


In [1150]:

# 'wind_speed' 값을 기반으로 'B' 변수를 만들기 위한 함수 정의
def categorize_wind_speed(speed):
    if speed < 0.3:
        return 0
    elif 0.3 <= speed < 1.5:
        return 1
    elif 1.5 <= speed < 3.3:
        return 2
    elif 3.3 <= speed < 5.5:
        return 3
    elif 5.5 <= speed < 8:
        return 4
    elif 8 <= speed < 10.8:
        return 5
    elif 10.8 <= speed < 13.9:
        return 6
    elif 13.9 <= speed < 17.2:
        return 7
    elif 17.2 <= speed < 20.7:
        return 8
    elif 20.7 <= speed < 24.5:
        return 9
    elif 24.5 <= speed < 28.4:
        return 10
    elif 28.4 <= speed < 32.6:
        return 11
    else:
        return 12

# 'wind_speed'를 기반으로 'B' 변수 생성
test['B'] = test['wind_speed'].apply(categorize_wind_speed)

# 결과 출력
test


,SAMPLE_ID,ARI_CO,ARI_PO,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,...,minute,weekday,SHIP_TYPE_CATEGORY_Cargo,SHIP_TYPE_CATEGORY_Container,SHIP_TYPE_CATEGORY_Tanker,wind_speed,weekend,day_초순,day_하순,B
0,TEST_000000,9.00000,26.00000,9.50814,2022-08-27 08:29:00,16803,20.00000,9,5510,10.00000,...,29,5,0,0,1,0.65521,1,0,1,1
1,TEST_000001,18.00000,22.00000,24.18668,2022-03-27 20:57:00,5714,40.00000,14,109000,20.00000,...,57,6,0,1,0,1.11759,1,0,1,1
2,TEST_000002,20.00000,14.00000,0.00000,2023-01-18 02:06:00,15156,20.00000,14,13700,10.00000,...,6,2,1,0,0,14.03807,0,0,0,7
3,TEST_000003,9.00000,26.00000,0.00000,2016-08-02 00:59:00,1306,10.00000,18,2840,10.00000,...,59,1,0,0,0,0.76195,0,1,0,1
4,TEST_000004,18.00000,22.00000,19.83419,2023-01-24 00:27:00,8819,30.00000,23,30700,20.00000,...,27,1,0,1,0,4.74102,0,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244984,TEST_244984,17.00000,103.00000,1.62570,2018-10-22 02:49:00,16386,10.00000,35,3120,10.00000,...,49,0,0,0,0,0.36454,0,0,1,1
244985,TEST_244985,17.00000,103.00000,30.64284,2019-06-08 20:00:00,14648,30.00000,17,46400,20.00000,...,0,5,0,0,0,0.00000,1,1,0,0
244986,TEST_244986,5.00000,69.00000,0.00000,2017-03-30 11:26:00,16878,20.00000,12,5280,10.00000,...,26,3,1,0,0,0.58762,0,0,1,1
244987,TEST_244987,24.00000,51.00000,0.00000,2022-08-10 07:20:00,5249,20.00000,27,7020,10.00000,...,20,2,0,1,0,2.47227,0,1,0,2


## 항구에 컨테이너선이 들어가면 보통 다 들어감. (2개 빼고) 
## 컨테이너선의 대기시간이 평균적으로 다른 것들보다 훨씬 작음
- MMW5
- TYY2

In [1151]:
# 'ARI_PO' 변수에서 특정 값들을 가지면 'Container_YN' 변수에 0을 할당, 아니면 1을 할당
specific_values = ['TMR7', 'YDP4', 'KSF1', 'SPG1', 'CXL1', 'IEW6', 'VFD8', 'PDL4', 'DMD4', 'EVL6', 'XIU1', 'EVX2', 'QGL7', 
                   'PPM2', 'LHD1', 'URD4', 'YFK5', 'FAV3', 'OKM4', 'CSP6', 'XGX5', 'GRQ5', 'HYV6', 'GQJ7', 'OOV8', 'UPI6',
                   'JEA6', 'QXE2', 'DIN2', 'ZME5', 'ONW1', 'XZF6', 'RGT8']

train['Container_YN'] = train['ARI_PO'].apply(lambda x: 0 if x in specific_values else 1)
test['Container_YN'] = test['ARI_PO'].apply(lambda x: 0 if x in specific_values else 1)

In [1152]:
train

,SAMPLE_ID,ARI_CO,ARI_PO,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,...,weekday,SHIP_TYPE_CATEGORY_Cargo,SHIP_TYPE_CATEGORY_Container,SHIP_TYPE_CATEGORY_Tanker,wind_speed,weekend,day_초순,day_하순,B,Container_YN
0,TRAIN_000000,4.00000,13.00000,30.73658,2020-10-15 04:03:00,24205,30.00000,28,73100,20.00000,...,3,0,0,0,3.77260,0,0,0,3,1
1,TRAIN_000001,4.00000,14.00000,63.22042,2019-09-17 02:55:00,19450,30.00000,15,37900,20.00000,...,1,0,1,0,7.42590,0,0,0,4,1
2,TRAIN_000002,4.00000,45.00000,90.42742,2019-02-23 06:43:00,20345,50.00000,7,115000,20.00000,...,5,0,1,0,0.00000,1,0,1,0,1
3,TRAIN_000003,8.00000,75.00000,0.00000,2020-09-18 22:06:00,1680,10.00000,33,1490,10.00000,...,4,1,0,0,8.50128,0,0,0,5,1
4,TRAIN_000004,16.00000,46.00000,8.81373,2022-08-13 12:57:00,2973,30.00000,10,27600,10.00000,...,5,0,1,0,3.00375,1,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367434,TRAIN_367434,4.00000,62.00000,55.40877,2022-06-16 14:27:00,3021,30.00000,16,26500,10.00000,...,3,0,0,0,3.60999,0,0,0,3,1
367435,TRAIN_367435,0.00000,37.00000,0.00000,2014-11-10 23:04:00,21011,40.00000,11,87300,20.00000,...,0,0,0,0,1.03774,0,1,0,1,1
367436,TRAIN_367436,4.00000,100.00000,59.01818,2017-11-11 22:23:00,9121,40.00000,13,93200,20.00000,...,5,0,0,0,0.37773,1,0,0,1,1
367437,TRAIN_367437,8.00000,66.00000,0.00000,2022-04-29 02:58:00,3755,10.00000,9,1280,0.00000,...,4,0,0,1,0.87207,0,0,1,1,1


In [1154]:
train.drop(['SAMPLE_ID','ATA'],axis=1,inplace=True)

## 이상치 데이터 처리

In [1156]:
from sklearn.ensemble import IsolationForest

# Isolation Forest 모델 생성
isol = IsolationForest(contamination=0.05)  # contamination 비율은 이상치 비율
isol.fit(train)

# 훈련 데이터에서 이상치 여부 판별
outlier_predictions_train = isol.predict(train)

# 테스트 데이터에서 이상치 여부 판별

# 이상치로 판별된 데이터 제거
train= train[outlier_predictions_train == 1]

# 제거된 이상치로 인해 줄어든 훈련 데이터 크기를 확인
print("훈련 데이터 크기 (이상치 제거 후):", train.shape)

훈련 데이터 크기 (이상치 제거 후): (344068, 37)


# ------------- 검증데이터 안나누고 모델링 해보기 -----------------------

In [1159]:
X = train.drop('CI_HOUR',axis=1)
y = train['CI_HOUR']
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

## 컨테이너인거 아닌거 모델링

In [910]:
X.drop(['SHIP_TYPE_CATEGORY_Tanker','SHIP_TYPE_CATEGORY_Cargo'],axis=1,inplace=True)

In [768]:
test.drop(['SHIP_TYPE_CATEGORY_Tanker','SHIP_TYPE_CATEGORY_Cargo'],axis=1,inplace=True)

In [779]:
# 'SHIP_TYPE_CATEGORY_Container' 변수 값이 1인 데이터셋
X_container_1 = X[X['SHIP_TYPE_CATEGORY_Container'] == 1]
y_container_1 = y[X['SHIP_TYPE_CATEGORY_Container'] == 1]

# 'SHIP_TYPE_CATEGORY_Container' 변수 값이 0인 데이터셋
X_container_0 = X[X['SHIP_TYPE_CATEGORY_Container'] == 0]
y_container_0 = y[X['SHIP_TYPE_CATEGORY_Container'] == 0]

# 'SHIP_TYPE_CATEGORY_Container' 변수 제거
X_container_1 = X_container_1.drop(columns=['SHIP_TYPE_CATEGORY_Container'])
X_container_0 = X_container_0.drop(columns=['SHIP_TYPE_CATEGORY_Container'])

# 1인 경우 모델 훈련
model_container_1 = xgb.XGBRegressor()
model_container_1.fit(X_container_1, y_container_1)

# 0인 경우 모델 훈련
model_container_0 = xgb.XGBRegressor()
model_container_0.fit(X_container_0, y_container_0)

# 전체 데이터셋에 대한 예측
predictions_container_1 = model_container_1.predict(X_container_1)
predictions_container_0 = model_container_0.predict(X_container_0)

# 예측값을 0 이상으로 조정
predictions_container_1 = [max(0, value) for value in predictions_container_1]
predictions_container_0 = [max(0, value) for value in predictions_container_0]

# 예측값을 다시 원래 인덱스에 맞게 변환
predictions_container_1_series = pd.Series(predictions_container_1, index=X_container_1.index)
predictions_container_0_series = pd.Series(predictions_container_0, index=X_container_0.index)

# 원래 데이터프레임에 예측값 추가
X['predicted_values'] = pd.concat([predictions_container_1_series, predictions_container_0_series])


mae_container = mean_absolute_error(y, X['predicted_values'])

mae_container


41.10457808603509

In [770]:
test.drop(['SAMPLE_ID','ATA'],axis=1,inplace=True)

In [774]:
import pandas as pd

# 'SHIP_TYPE_CATEGORY_Container' 변수 값이 1인 데이터셋
test_container_1 = test[test['SHIP_TYPE_CATEGORY_Container'] == 1]
test_container_0 = test[test['SHIP_TYPE_CATEGORY_Container'] == 0]

# 'SHIP_TYPE_CATEGORY_Container' 변수 제거
test_container_1 = test_container_1.drop(columns=['SHIP_TYPE_CATEGORY_Container'])
test_container_0 = test_container_0.drop(columns=['SHIP_TYPE_CATEGORY_Container'])

# 전체 데이터셋에 대한 예측
predictions_container_1 = model_container_1.predict(test_container_1)
predictions_container_0 = model_container_0.predict(test_container_0)

# 예측값을 0 이상으로 조정
predictions_container_1 = [max(0, value) for value in predictions_container_1]
predictions_container_0 = [max(0, value) for value in predictions_container_0]

# 예측값을 다시 원래 인덱스에 맞게 변환
predictions_container_1_series = pd.Series(predictions_container_1, index=test_container_1.index)
predictions_container_0_series = pd.Series(predictions_container_0, index=test_container_0.index)

# 원래 데이터프레임에 예측값 추가
test['predicted_values'] = pd.concat([predictions_container_1_series, predictions_container_0_series])

# 예측값을 0 이상으로 조정
test['predicted_values'] = test['predicted_values'].apply(lambda x: max(0, x))

# 결과 확인
test.head()


,ARI_CO,ARI_PO,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,...,minute,weekday,SHIP_TYPE_CATEGORY_Container,wind_speed,weekend,day_초순,day_하순,B,Container_YN,predicted_values
0,9.00000,26.00000,9.50814,16803,20.00000,9,5510,10.00000,10.00000,3720,...,29,5,0,0.65521,1,0,1,1,1,88.72729
1,18.00000,22.00000,24.18668,5714,40.00000,14,109000,20.00000,10.00000,94400,...,57,6,1,1.11759,1,0,1,1,1,260.55817
2,20.00000,14.00000,0.00000,15156,20.00000,14,13700,10.00000,10.00000,8930,...,6,2,0,14.03807,0,0,0,7,1,6.86846
3,9.00000,26.00000,0.00000,1306,10.00000,18,2840,10.00000,10.00000,1970,...,59,1,0,0.76195,0,1,0,1,1,2.57195
4,18.00000,22.00000,19.83419,8819,30.00000,23,30700,20.00000,10.00000,26000,...,27,1,1,4.74102,0,0,1,3,1,140.25519


## 그냥 모델링

In [1160]:
# 필요한 라이브러리 불러오기
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

# XGBoost 회귀 모델 생성
model = xgb.XGBRegressor()

# 모델 훈련
model.fit(X,y)


# 테스트 세트에 대한 예측
predictions = model.predict(X)

# 예측값을 0 이상으로 조정
predictions = [max(0, value) for value in predictions]
# MAE 계산
mae = mean_absolute_error(y, predictions)

print("Mean Absolute Error (MAE):", mae)


Mean Absolute Error (MAE): 40.08202001369564


In [913]:
# 학습된 XGBoost 모델로부터 변수 중요도를 얻기
importances = model.feature_importances_

# 중요도가 높은 순으로 변수의 인덱스를 정렬
indices = np.argsort(importances)[::-1]


In [914]:
# 변수중요도 밑에 컬럼 순서로 보면될듯. LENGTH , 더미 Container, PORT_SIZE 순으로 나옴
indices

array([ 2, 27, 26,  0, 21, 22, 20,  6, 23,  9, 18, 19, 17, 10,  1, 31, 24,
       12,  4, 30, 16,  5, 15,  7, 11,  8, 13, 14,  3, 25, 32, 28, 29, 33])

In [515]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 362177 entries, 0 to 367439
Data columns (total 35 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   ARI_CO                        362177 non-null  float64
 1   ARI_PO                        362177 non-null  float64
 2   DIST                          362177 non-null  float64
 3   ID                            362177 non-null  int64  
 4   BREADTH                       362177 non-null  float64
 5   BUILT                         362177 non-null  int64  
 6   DEADWEIGHT                    362177 non-null  int64  
 7   DEPTH                         362177 non-null  float64
 8   DRAUGHT                       362177 non-null  float64
 9   GT                            362177 non-null  int64  
 10  LENGTH                        362177 non-null  float64
 11  SHIPMANAGER                   362177 non-null  int64  
 12  FLAG                          362177 non-nul

# 위 모델 test로 돌렸을 때 46나옴

In [563]:
test.describe()

,ARI_CO,ARI_PO,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,...,hour,minute,weekday,SHIP_TYPE_CATEGORY_Cargo,SHIP_TYPE_CATEGORY_Container,SHIP_TYPE_CATEGORY_Tanker,wind_speed,weekend,B,Container_YN
count,244989.00000,244989.00000,244989.00000,244989.00000,244989.00000,244989.00000,244989.00000,244989.00000,244989.00000,244989.00000,...,244989.00000,244989.00000,244989.00000,244989.00000,244989.00000,244989.00000,244989.00000,244989.00000,244989.00000,244989.00000
mean,9.21381,45.82269,19.32316,12173.80415,30.53113,14.75261,63471.09756,16.46466,11.08531,41266.17587,...,11.84275,31.08889,2.40620,0.10405,0.42421,0.05247,2.69465,0.13652,1.99035,1.00000
std,6.39196,28.72925,30.94568,7223.55732,10.94169,7.06426,63381.42858,6.03980,4.58897,39187.09136,...,6.64437,18.28569,1.82110,0.30533,0.49422,0.22297,2.81619,0.34334,1.38218,0.00000
min,1.00000,1.00000,0.00000,0.00000,10.00000,0.00000,100.00000,0.00000,0.00000,150.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000
25%,5.00000,22.00000,0.00000,5848.00000,20.00000,10.00000,14100.00000,10.00000,10.00000,10000.00000,...,7.00000,15.00000,1.00000,0.00000,0.00000,0.00000,0.72993,0.00000,1.00000,1.00000
50%,7.00000,40.00000,6.90063,12268.00000,30.00000,13.00000,47200.00000,20.00000,10.00000,30500.00000,...,12.00000,32.00000,2.00000,0.00000,0.00000,0.00000,1.33959,0.00000,1.00000,1.00000
75%,17.00000,67.00000,25.63768,18491.00000,40.00000,18.00000,81500.00000,20.00000,10.00000,51900.00000,...,17.00000,48.00000,4.00000,0.00000,1.00000,0.00000,4.04500,0.00000,3.00000,1.00000
max,25.00000,103.00000,199.99750,24630.00000,70.00000,71.00000,404000.00000,30.00000,20.00000,237000.00000,...,23.00000,59.00000,6.00000,1.00000,1.00000,1.00000,31.24994,1.00000,11.00000,1.00000


In [776]:
submission = pd.read_csv('/Users/idogeon/Desktop/현대/데이터/sample_submission.csv')

In [777]:
submission['CI_HOUR'] = test['predicted_values']
submission

,SAMPLE_ID,CI_HOUR
0,TEST_000000,88.72729
1,TEST_000001,260.55817
2,TEST_000002,6.86846
3,TEST_000003,2.57195
4,TEST_000004,140.25519
...,...,...
244984,TEST_244984,131.04356
244985,TEST_244985,836.05347
244986,TEST_244986,17.80770
244987,TEST_244987,117.18302


In [778]:
submission.to_csv('submission1.csv',index=False)